In [289]:
import pandas as pd
import numpy as np
import xlsxwriter
from datetime import datetime
from xlsxwriter.utility import xl_rowcol_to_cell, xl_cell_to_rowcol
from collections import OrderedDict

# Setup

## set up workbook 

In [290]:
workbook = xlsxwriter.Workbook('IslandEnergy.xlsx')
ws_const_in = workbook.add_worksheet(name="Construction Period Cost Inputs")
ws_const_calc = workbook.add_worksheet(name="Construction Period Calcs")

# dictionary of references 
refd= {}
refd["sheetnames"] = list(workbook.sheetnames.keys())
refd["gen_startrow"] = 4 

## define formats

In [291]:
# Add a bold format to use to highlight cells.
bold_format = workbook.add_format({'bold': True})
bold_format.set_text_wrap()

# Add a number format for cells with money.
money_format = workbook.add_format({'num_format': '$#,##0'})

# create format to use in merged range 
merge_format = workbook.add_format({
    'bold': 1,
    # 'border': 1,
    # 'align': 'center',
    # 'valign': 'vcenter',
    # 'fg_color': 'yellow'
    })

percent_format = workbook.add_format({'num_format': '0.00%'})

left_align_format = workbook.add_format()
left_align_format.set_align("left")

## worksheet formatting 

In [292]:
# Page Headers 
for name, sheet in workbook.sheetnames.items():
    sheet.write('A1', 'Project Name', bold_format)
    sheet.write('A2', 'Sheet Name', bold_format)
    sheet.merge_range('B1:C1', "Island Energy", bold_format)
    sheet.merge_range('B2:C2', name, bold_format)
    sheet.set_column("A:Z", 30)

# Inputs 

## construction period inputs

In [293]:
months = range(1,25)
# data 
# payments at financial close  
# TODO put these in a dif file 
fc_costs = {
    "Land": 12.5e6,
    "Development Expenses": 28.5e6,
    "Development Fees": 14.7e6,
    "Closing Costs" : 14.392e6,
    "Contingency Fees": 30.754e6
}

# eng, procurement, const base cost 
epc_cost = 455e6

# other payments over course of construction period 
split_costs = {
"Interest and Fees" : 56.853e6,
"Insurance and Fees" : 25e6,
"Management and Oversight ": 8.15e6
}


epc_sched = OrderedDict()

for i in list(months):
    if i <= 3:
        epc_sched[i] = 0.02
    elif i <= 8: 
        epc_sched[i] = 0.04
    elif i <= 14: 
        epc_sched[i] = 0.06
    elif i <= 18: 
        epc_sched[i] = 0.05
    else: 
        epc_sched[i] = 0.03


In [294]:
# costs paid at financial close 
ws_const_in.merge_range('A4:B4', 'Costs Paid at Financial Close', merge_format)

refd["cpi"] = {}
refd["cpi"]["heading_row"] = xl_cell_to_rowcol('A4') 


row = refd["cpi"]["heading_row"][0]+1
refd["cpi"]["fc_sr"] = row
col = 0
for k,v in fc_costs.items():
    ws_const_in.write(row, col, k)
    ws_const_in.write(row, col+1, v, money_format)
    row += 1



In [295]:
# costs paid over construction period 

# skip a row to make header 
row+=1
ws_const_in.merge_range(row, 0, row, 1, 'Costs Paid Over Construction Period', merge_format)

row+=1
refd["cpi"]["epc_sr"] = row 
ws_const_in.write(row, col, "EPC Cost")
ws_const_in.write(row, col+1, epc_cost)

row+=1
refd["cpi"]["split_sr"] = row
for k,v in split_costs.items():
    ws_const_in.write(row, col, k)
    ws_const_in.write(row, col+1, v, money_format)
    row += 1

In [296]:
# skip a row to make header 
row+=1
ws_const_in.merge_range(row, 0, row, 1, 'EPC Payment Schedule', merge_format)

row+=1
ws_const_in.write(row, 0, "EPC Contract Month", bold_format)
ws_const_in.write(row, 1, "Max. Payment, as percentage of total EPC price", bold_format)

row+=1
refd["cpi"]["epc_perc_sr"] = row
for k,v in epc_sched.items():
    ws_const_in.write(row, 0, k)
    ws_const_in.write(row, 1, v, )#percent_format
    row += 1



# Calculations 

## construction calculations 

In [297]:

# headings 
headings = ["Payment Schedule Category", "Payment Name", "Financial Close"] + list(months) # TOOD -> make this nice, month 1 + date 

col=0
for heading in headings:
    ws_const_calc.write(refd["gen_startrow"], col, heading, bold_format)
    col +=1


In [298]:
def calc_on_inputs(local_start_row, foreign_start_row, input_data, merge_name, eqn_start_col, calc_fx):
    # costs paid at financial close 
    len_data = len(list(input_data.keys()))

    # merge names in the first colum 
    if len_data > 1: 
        sheet.merge_range(local_start_row, 0, local_start_row+len_data-1, 0, merge_name, bold_format)
    else:
        sheet.write(local_start_row, 0, merge_name, bold_format)
        

    # local columns
    name_col = 1
    row = local_start_row
    # for iterating over the foreign sheet
    i=0 

    for cost in range(len_data):
        # get cell name from other sheet 
        name_cell = xl_rowcol_to_cell(foreign_start_row+i, 0)
        # write heading 
        ws_const_calc.write_formula(row, name_col, f"='{refd['sheetnames'][0]}'!{name_cell}", left_align_format) #TODO change sheetname to be more broad later 

        # do monthly calculation 
        calc_fx(foreign_start_row, eqn_start_col, row, i)

        # iterate over foreign and local sheet 
        row+=1
        i+=1

    return row # return ending row 

In [299]:
# monthly plit costs 
def calc_split(foreign_start_row, eqn_start_col, row, i): 
        # nothing paid at financial close 
        ws_const_calc.write_formula(row, eqn_start_col, "=0", money_format)
        # do monthly calculation 
        for month in months:
            # get cell name from other sheet 
            data_cell = xl_rowcol_to_cell(foreign_start_row+i, 1)
            # do calculation 
            ws_const_calc.write_formula(row, eqn_start_col+month, f"='{refd['sheetnames'][0]}'!{data_cell}/24", money_format)

In [300]:
# EPC split costs 
def calc_epc(foreign_start_row, eqn_start_col, row, i):
        epc_sched_row = refd["cpi"]["epc_perc_sr"]
        # nothing paid at financial close 
        ws_const_calc.write_formula(row, eqn_start_col, "=0", money_format)
        # do monthly calculation 
        ci_sheet = refd['sheetnames'][0]
        for ix, month in enumerate(months):
            # get cell name from other sheet 
            epc_percent_cell = xl_rowcol_to_cell(epc_sched_row+ix, 1)
            epc_payment_cell = xl_rowcol_to_cell(foreign_start_row, 1)
            print(epc_percent_cell, epc_payment_cell, ix)
            # do calculation 
            ws_const_calc.write_formula(row, eqn_start_col+month, 
            f"='{ci_sheet}'!{epc_payment_cell} * '{ci_sheet}'!{epc_percent_cell}", money_format)
            # ws_const_calc.write_formula(row, eqn_start_col+month, f"=C9*C10", money_format)

In [301]:
"='Construction Period Cost Inputs'!B12*'Construction Period Cost Inputs'!B19"

"='Construction Period Cost Inputs'!B12*'Construction Period Cost Inputs'!B19"

In [302]:
# costs paid at financial close 
def calc_fc(foreign_start_row, eqn_start_col, row, i):
    # get cell name from other sheet 
    data_cell = xl_rowcol_to_cell(foreign_start_row+i, 1)
    ws_const_calc.write_formula(row, eqn_start_col, f"='{refd['sheetnames'][0]}'!{data_cell}", money_format)
    # enter 0s for everything ele 
    for month in months:
        ws_const_calc.write_formula(row, eqn_start_col+month, "=0", money_format)

In [303]:
end_row = calc_on_inputs(refd["gen_startrow"]+1, refd["cpi"]["split_sr"], split_costs, "Monthly Payments", 2, calc_split)

er = calc_on_inputs(end_row, refd["cpi"]["fc_sr"], fc_costs, "Payments at Financial Close", 2, calc_fc)

# calc_on_inputs(local_start_row, foreign_start_row, input_data, merge_name, eqn_start_col, calc_fx)
epc_data = {
    "1": 1
}
er_epc = calc_on_inputs(er, refd["cpi"]["epc_sr"], epc_data, "EPC payments", 2, calc_epc)


B19 B12 0
B20 B12 1
B21 B12 2
B22 B12 3
B23 B12 4
B24 B12 5
B25 B12 6
B26 B12 7
B27 B12 8
B28 B12 9
B29 B12 10
B30 B12 11
B31 B12 12
B32 B12 13
B33 B12 14
B34 B12 15
B35 B12 16
B36 B12 17
B37 B12 18
B38 B12 19
B39 B12 20
B40 B12 21
B41 B12 22
B42 B12 23


# Close

In [304]:
workbook.close()